In [90]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import  pandas as pd
import numpy as np
import os

In [91]:
## load the trained model
model = load_model("../model.h5")

In [92]:
## load the encoders and scaler 
import pickle

with open("../label_encoder_gender.pkl", "rb") as file:
    lable_encoder_gender = pickle.load(file)
with open("../onehot_encoder_geo.pkl", "rb") as file:
    lable_encoder_geo = pickle.load(file)
with open ("../scaler.pkl",'rb') as file :
    scaler = pickle.load(file)


In [93]:
input_data = {
'CreditScore': 600,
'Geography': 'France',
'Gender': 'Male',
'Age': 40,
'Tenure': 3,
'Balance': 60000,
'NumOfProducts': 2,
'HasCrCard': 1,
'IsActiveMember': 1,
'EstimatedSalary': 5000,
}



In [94]:
print("OneHot Encoder Categories:", lable_encoder_geo.categories_)
print("Gender Label Encoder Classes:", lable_encoder_gender.classes_)

OneHot Encoder Categories: [array(['France', 'Germany', 'Spain'], dtype=object)]
Gender Label Encoder Classes: ['Female' 'Male']


In [95]:
# Correctly transform Geography with OneHotEncoder
geo_encoded = lable_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=lable_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/udmuser/Desktop/ANN/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [96]:
input_df = pd.DataFrame([input_data])  
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,5000


In [97]:
## encode categorial value
input_df['Gender'] = lable_encoder_gender.transform(input_df['Gender'])

In [98]:
## concatination one hot encoded 
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)


In [102]:
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,5000,1.0,0.0,0.0


In [104]:
## sacling the data 
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.65923237,  1.00150113,
        -0.57946723, -0.57638802]])

In [111]:
## predictchurn 
prediction = model.predict(input_scaled)
prediction[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


np.float32(0.015872234)